# **Customer Credit Card Spend Prediction Model**

**Link to Competition: https://datahack.analyticsvidhya.com/contest/amexpert-decipher-women-machine-learning-hackathon/  **

**Leaderboard**: My Username in the competition - anonymousC683HQ

https://datahack.analyticsvidhya.com/contest/amexpert-decipher-women-machine-learning-hackathon/pvt_lb

**Client Description**

American Express is a globally integrated payments company, providing customers with access to products, insights and experiences that enrich lives and build business success. It is
* One of the world's most recognizable brands
* An exceptionally strong heritage - over 160 years of leadership and reinvention
* World's largest card issuer by purchase volume
* Recognized as the most innovative company in our industry
* Cutting-edge Information Management and unique Closed-Loop network
* Customer loyalty experts with industry-leading rewards programs and platforms
* A rich history and a bright future

**Objective**

To predict the credit card spend of customers for next three months using the past three months customer usage and demographic data. 

**Evaluation Criteria**

Accuracy of the predictive model is evaluated using Root Mean Squared Logarithmic Error (RMSLE). 
                          
**Data**

Customer credit/ debit usage and demographic information for three months – Apr, May, Jun – is shared for training. 
* Rows – 32,820
* Columns - 44

A new set of customers’ data is shared for testing the model performance. Evaluation is based on the accuracy achieved by testing the model on test dataset.
* Rows – 14,067
* Columns – 43 (Except the target variable)


**Approach**

1)  Data Ingestion and Cleaning

* Replaced all numeric rows containing NA with 0. Reason for imputing with 0 is that it made sense to retain 0 rather than imputing with mean/ median for most variables. Ex: Credit/ Debit card spend/ count may be 0 in real world scenarios.
*  Target variable ‘cc_cons’ is found to be extremely skewed to the right. That is, very few customers had spent extremely high compared to others. 
    * However, model finds it really difficult to predict for low spend customers rather than high spend customers. 
    * This is found after implementing the model and analyzing the error terms. 
    * Model predicted higher values for extremely low spenders that increased the final RMSLE.
    * Hence, replaced those ‘cc_cons’ values that fall under 1% percentile with median value to help the model predict lesser values for such cases and it helped to boost the accuracy.
* Created an exhaustive list of 157 derived variables to capture the interaction among independent variables.
* Label encoded categorical variables to help for modeling exercise.

2) Modeling:

* Used LightGBM for predicting credit card spend after trying out multiple other algorithms like XGBoost and CatBoost.
* Fine-tuned the parameters of LightGBM using Grid Search and Random Search to find out the best parameters for fitting.
* Performed 10 folds cross-validation to remove any bias and avoid over-fitting.


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import lightgbm as lgb
import time

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['amexpert-test-dataset', 'amexpert-dataset']


**Data Extraction and Pre-processing**

In [2]:
## Data import
train_data = pd.read_csv("../input/amexpert-dataset/train.csv")
test_data = pd.read_csv("../input/amexpert-test-dataset/test_9K3DBWQ.csv")

In [3]:
## Dimensions of dataset
train_data.shape
test_data.shape

(32820, 44)

In [5]:
## Combining train and test dataset to do same manipulation for both dataset
test_data["cc_cons"] = np.nan
train_data['Flag'] = 'Train'
test_data['Flag'] = 'Test'

In [9]:
## Replacing values that fall under less than 1% percentile in 'cc_cons' with median
print(np.quantile(train_data.cc_cons,[0.01])[0])
print(train_data.cc_cons.median())

train_data.loc[train_data['cc_cons'] < np.quantile(train_data.cc_cons,[0.01])[0],"cc_cons"] = train_data.cc_cons.median()

print(np.quantile(train_data.cc_cons,[0.01])[0])
print(train_data.cc_cons.median())

138.19
3141.0
200.0
3141.0


In [10]:
## Combining train and test dataset to do same manipulation for both dataset
data = pd.concat([train_data, test_data])

In [11]:
## Replacing numeric columns containing NA with 0
for column in data.select_dtypes(include=np.number).columns:
    data[column] = data[column].fillna(0)

In [12]:
## Replacing NA rows with 'N' for loan_enq column
data["loan_enq"] = data["loan_enq"].fillna("N")

**Feature Engineering**

In [14]:
## Dervied variables creation
data["cc_apr_may_spend"] = np.where((data["cc_cons_apr"]==0) & (data["cc_cons_may"]!=0), 1,
                                    np.where((data["cc_cons_apr"]==0) & (data["cc_cons_may"]==0), 0,
                                             round((data["cc_cons_may"]-data["cc_cons_apr"])/data["cc_cons_apr"],2)))
data["cc_may_jun_spend"] = np.where((data["cc_cons_may"]==0) & (data["cc_cons_jun"]!=0), 1,
                                    np.where((data["cc_cons_may"]==0) & (data["cc_cons_jun"]==0), 0,
                                    round((data["cc_cons_jun"]-data["cc_cons_may"])/data["cc_cons_may"],2)))
data["dc_apr_may_spend"] = np.where((data["dc_cons_apr"]==0) & (data["dc_cons_may"]!=0), 1,
                                    np.where((data["dc_cons_apr"]==0) & (data["dc_cons_may"]==0), 0,
                                             round((data["dc_cons_may"]-data["dc_cons_apr"])/data["dc_cons_apr"],2)))
data["dc_may_jun_spend"] = np.where((data["dc_cons_may"]==0) & (data["dc_cons_jun"]!=0), 1,
                                    np.where((data["dc_cons_may"]==0) & (data["dc_cons_jun"]==0), 0,
                                             round((data["dc_cons_jun"]-data["dc_cons_may"])/data["dc_cons_may"],2)))
data["cc_count_apr_may"] = np.where((data["cc_count_apr"]==0) & (data["cc_count_may"]!=0), 1,
                                    np.where((data["cc_count_apr"]==0) & (data["cc_count_may"]==0), 0,
                                             round((data["cc_count_may"]-data["cc_count_apr"])/data["cc_count_apr"],2)))
data["cc_count_may_jun"] = np.where((data["cc_count_may"]==0) & (data["cc_count_jun"]!=0), 1,
                                    np.where((data["cc_count_may"]==0) & (data["cc_count_jun"]==0), 0,
                                             round((data["cc_count_jun"]-data["cc_count_may"])/data["cc_count_may"],2)))
data["dc_count_apr_may"] = np.where((data["dc_count_apr"]==0) & (data["dc_count_may"]!=0), 1,
                                    np.where((data["dc_count_apr"]==0) & (data["dc_count_may"]==0), 0,
                                             round((data["dc_count_may"]-data["dc_count_apr"])/data["dc_count_apr"],2)))
data["dc_count_may_jun"] = np.where((data["dc_count_may"]==0) & (data["dc_count_jun"]!=0), 1,
                                    np.where((data["dc_count_may"]==0) & (data["dc_count_jun"]==0), 0,
                                             round((data["dc_count_jun"]-data["dc_count_may"])/data["dc_count_may"],2)))
data["debit_amount_apr_may"] = np.where((data["debit_amount_apr"]==0) & (data["debit_amount_may"]!=0), 1,
                                        np.where((data["debit_amount_apr"]==0) & (data["debit_amount_may"]==0), 0,
                                                 round((data["debit_amount_may"]-data["debit_amount_apr"])/data["debit_amount_apr"],2)))
data["debit_amount_may_jun"] = np.where((data["debit_amount_may"]==0) & (data["debit_amount_jun"]==0), 1,
                                        np.where((data["debit_amount_may"]==0) & (data["debit_amount_jun"]==0), 0,
                                                 round((data["debit_amount_jun"]-data["debit_amount_may"])/data["debit_amount_may"],2)))
data["credit_amount_apr_may"] = np.where((data["credit_amount_apr"]==0) & (data["credit_amount_may"]!=0), 1,
                                         np.where((data["credit_amount_apr"]==0) & (data["credit_amount_may"]==0), 0,
                                                  round((data["credit_amount_may"]-data["credit_amount_apr"])/data["credit_amount_apr"],2)))
data["credit_amount_may_jun"] = np.where((data["credit_amount_may"]==0) & (data["credit_amount_jun"]!=0), 1,
                                         np.where((data["credit_amount_may"]==0) & (data["credit_amount_jun"]==0), 0,
                                                  round((data["credit_amount_jun"]-data["credit_amount_may"])/data["credit_amount_may"],2)))
data["debit_count_apr_may"] = np.where((data["debit_count_apr"]==0) & (data["debit_count_may"]!=0), 1,
                                       np.where((data["debit_count_apr"]==0) & (data["debit_count_may"]==0), 0,
                                                round((data["debit_count_may"]-data["debit_count_apr"])/data["debit_count_apr"],2)))
data["debit_count_may_jun"] = np.where((data["debit_count_may"]==0) & (data["debit_count_jun"]!=0), 1,
                                       np.where((data["debit_count_may"]==0) & (data["debit_count_jun"]==0), 0,
                                                round((data["debit_count_jun"]-data["debit_count_may"])/data["debit_count_may"],2)))
data["credit_count_apr_may"] = np.where((data["credit_count_apr"]==0) & (data["credit_count_may"]==0), 1,
                                        np.where((data["credit_count_apr"]==0) & (data["credit_count_may"]==0), 0,
                                        round((data["credit_count_may"]-data["credit_count_apr"])/data["credit_count_apr"],2)))
data["credit_count_may_jun"] = np.where((data["credit_count_may"]==0) & (data["credit_count_jun"]!=0), 1,
                                        np.where((data["credit_count_may"]==0) & (data["credit_count_jun"]==0), 0,
                                                 round((data["credit_count_jun"]-data["credit_count_may"])/data["credit_count_may"],2)))
data["max_credit_amount_apr_may"] = np.where((data["max_credit_amount_apr"]==0) & (data["max_credit_amount_may"]!=0), 1,
                                             np.where((data["max_credit_amount_apr"]==0) & (data["max_credit_amount_may"]==0), 0,
                                                      round((data["max_credit_amount_may"]-data["max_credit_amount_apr"])/data["max_credit_amount_apr"],2)))
data["max_credit_amount_may_jun"] = np.where((data["max_credit_amount_may"]==0) & (data["max_credit_amount_jun"]!=0), 1,
                                             np.where((data["max_credit_amount_may"]==0) & (data["max_credit_amount_jun"]==0), 0,
                                                      round((data["max_credit_amount_jun"]-data["max_credit_amount_may"])/data["max_credit_amount_may"],2)))

data["avg_trend_cc_spend"] = round((data["cc_apr_may_spend"] + data["cc_may_jun_spend"])/2,2)
data["avg_trend_dc_spend"] = round((data["dc_apr_may_spend"] + data["dc_may_jun_spend"])/2,2)
data["avg_trend_cc_count"] = round((data["cc_count_apr_may"] + data["cc_count_may_jun"])/2,2)
data["avg_trend_dc_count"] = round((data["dc_count_apr_may"] + data["dc_count_may_jun"])/2,2)
data["avg_trend_debit_amount"] = round((data["debit_amount_apr_may"] + data["debit_amount_may_jun"])/2,2)
data["avg_trend_credit_amount"] = round((data["credit_amount_apr_may"] + data["credit_amount_may_jun"])/2,2)
data["avg_trend_debit_count"] = round((data["debit_count_apr_may"] +data["debit_count_may_jun"])/2,2)
data["avg_trend_credit_count"] = round((data["credit_count_apr_may"] + data["credit_count_may_jun"])/2,2)
data["avg_trend_max_credit_amount"] = round((data["max_credit_amount_apr_may"] + data["max_credit_amount_may_jun"])/2,2)

data["total_cc_spend"] = data["cc_cons_apr"]+data["cc_cons_may"]+data["cc_cons_jun"]
data["avg_cc_spend_per_mnth"] = (data["cc_cons_apr"]+data["cc_cons_may"]+data["cc_cons_jun"])/3
data["total_dc_spend"] = data["dc_cons_apr"]+data["dc_cons_may"]+data["dc_cons_jun"]
data["avg_dc_spend_per_mnth"] = (data["dc_cons_apr"]+data["dc_cons_may"]+data["dc_cons_jun"])/3
data["perc_cc_spend"] = np.where((round(data["total_cc_spend"]/(data["total_cc_spend"]+data["total_dc_spend"]),2)).isna(),0,
                                 round(data["total_cc_spend"]/(data["total_cc_spend"]+data["total_dc_spend"]),2))
data["perc_dc_spend"] = np.where((round(data["total_dc_spend"]/(data["total_cc_spend"]+data["total_dc_spend"]),2)).isna(),0,
                                 round(data["total_dc_spend"]/(data["total_cc_spend"]+data["total_dc_spend"]),2))
data["total_dc_count"] = data["dc_count_apr"]+data["dc_count_may"]+data["dc_count_jun"]
data["avg_dc_count_per_mnth"] = (data["dc_count_apr"]+data["dc_count_may"]+data["dc_count_jun"])/3
data["total_cc_count"] = data["cc_count_apr"]+data["cc_count_may"]+data["cc_count_jun"]
data["avg_cc_count_per_mnth"] = (data["cc_count_apr"]+data["cc_count_may"]+data["cc_count_jun"])/3
data["perc_cc_count"] = np.where((round(data["total_cc_count"]/(data["total_cc_count"]+data["total_dc_count"]),2)).isna(),0,
                                 round(data["total_cc_count"]/(data["total_cc_count"]+data["total_dc_count"]),2))
data["perc_dc_count"] = np.where((round(data["total_dc_count"]/(data["total_cc_count"]+data["total_dc_count"]),2)).isna(),0,
                                 round(data["total_dc_count"]/(data["total_cc_count"]+data["total_dc_count"]),2))
data["total_debit_amount"] = data["debit_amount_apr"]+data["debit_amount_may"]+data["debit_amount_jun"]
data["avg_debit_amount_per_mnth"] = (data["debit_amount_apr"]+data["debit_amount_may"]+data["debit_amount_jun"])/3
data["total_credit_amount"] = data["credit_amount_apr"]+data["credit_amount_may"]+data["credit_amount_jun"]
data["avg_credit_amount_per_mnth"] = (data["credit_amount_apr"]+data["credit_amount_may"]+data["credit_amount_jun"])/3
data["perc_credit_amount"] = np.where((round(data["total_credit_amount"]/(data["total_credit_amount"]+data["total_debit_amount"]),2)).isna(),0,
                                 round(data["total_credit_amount"]/(data["total_credit_amount"]+data["total_debit_amount"]),2))
data["perc_debit_amount"] = np.where((round(data["total_debit_amount"]/(data["total_credit_amount"]+data["total_debit_amount"]),2)).isna(),0,
                                 round(data["total_debit_amount"]/(data["total_credit_amount"]+data["total_debit_amount"]),2))
data["total_debit_count"] = data["debit_count_apr"]+data["debit_count_may"]+data["debit_count_jun"]
data["avg_debit_count_per_mnth"] = (data["debit_count_apr"]+data["debit_count_may"]+data["debit_count_jun"])/3
data["total_credit_count"] = data["credit_count_apr"]+data["credit_count_may"]+data["credit_count_jun"]
data["avg_credit_count_per_mnth"] = (data["credit_count_apr"]+data["credit_count_may"]+data["credit_count_jun"])/3
data["perc_credit_count"] = np.where((round(data["total_credit_count"]/(data["total_credit_count"]+data["total_debit_count"]),2)).isna(),0,
                                 round(data["total_credit_count"]/(data["total_credit_count"]+data["total_debit_count"]),2))
data["perc_debit_count"] = np.where((round(data["total_debit_count"]/(data["total_credit_count"]+data["total_debit_count"]),2)).isna(),0,
                                 round(data["total_debit_count"]/(data["total_credit_count"]+data["total_debit_count"]),2))

data["avg_max_credit_amount_per_mnth"] = (data["max_credit_amount_apr"]+data["max_credit_amount_may"]+data["max_credit_amount_jun"])/3    
   
data["total_loan_active"] = data["personal_loan_active"] + data["vehicle_loan_active"]
data["total_loan_closed"] = data["personal_loan_closed"] + data["vehicle_loan_closed"]
data["perc_personal_loan_active"] = np.where((round(data["personal_loan_active"] / data["total_loan_active"], 2)).isna(), 0,
                                             round(data["personal_loan_active"] / data["total_loan_active"], 2))
data["perc_vehicle_loan_active"] = np.where((round(data["vehicle_loan_active"]/ data["total_loan_active"], 2)).isna(), 0 ,
                                            round(data["vehicle_loan_active"]/ data["total_loan_active"], 2))
data["avg_emi_per_active_loan"] = np.where((round(data["emi_active"]/data["total_loan_active"],2)).isna(),0,
                                           round(data["emi_active"]/data["total_loan_active"],2))
                                           
data["total_investment"] = data["investment_1"]+data["investment_2"]+data["investment_3"]+data["investment_4"]
data["perc_inv1"] = np.where((round(data["investment_1"]/data["total_investment"],2)).isna(), 0,
                             round(data["investment_1"]/data["total_investment"],2))
data["perc_inv2"] = np.where((round(data["investment_2"]/data["total_investment"],2)).isna(), 0,
                             round(data["investment_2"]/data["total_investment"],2))
data["perc_inv3"] = np.where((round(data["investment_3"]/data["total_investment"],2)).isna(), 0,
                             round(data["investment_3"]/data["total_investment"],2))
data["perc_inv4"] = np.where((round(data["investment_4"]/data["total_investment"],2)).isna(), 0,
                             round(data["investment_4"]/data["total_investment"],2))

data["perc_cc_spend_apr"] = np.where((round(data["cc_cons_apr"]/(data["cc_cons_apr"]+data["dc_cons_apr"]),2)).isna(),0,(round(data["cc_cons_apr"]/(data["cc_cons_apr"]+data["dc_cons_apr"]),2)))
data["perc_cc_spend_may"] = np.where((round(data["cc_cons_may"]/(data["cc_cons_may"]+data["dc_cons_may"]),2)).isna(),0,(round(data["cc_cons_may"]/(data["cc_cons_may"]+data["dc_cons_may"]),2)))
data["perc_cc_spend_jun"] = np.where((round(data["cc_cons_jun"]/(data["cc_cons_jun"]+data["dc_cons_jun"]),2)).isna(),0,(round(data["cc_cons_jun"]/(data["cc_cons_jun"]+data["dc_cons_jun"]),2)))

data["perc_dc_spend_apr"] = np.where((round(data["dc_cons_apr"]/(data["cc_cons_apr"]+data["dc_cons_apr"]),2)).isna(),0,(round(data["dc_cons_apr"]/(data["cc_cons_apr"]+data["dc_cons_apr"]),2)))
data["perc_dc_spend_may"] = np.where((round(data["dc_cons_may"]/(data["cc_cons_may"]+data["dc_cons_may"]),2)).isna(),0,(round(data["dc_cons_may"]/(data["cc_cons_may"]+data["dc_cons_may"]),2)))
data["perc_dc_spend_jun"] = np.where((round(data["dc_cons_jun"]/(data["cc_cons_jun"]+data["dc_cons_jun"]),2)).isna(),0,(round(data["dc_cons_jun"]/(data["cc_cons_jun"]+data["dc_cons_jun"]),2)))

data["perc_cc_count_apr"] = np.where((round(data["cc_count_apr"]/(data["cc_count_apr"]+data["dc_count_apr"]),2)).isna(),0,(round(data["cc_count_apr"]/(data["cc_count_apr"]+data["dc_count_apr"]),2)))
data["perc_cc_count_may"] = np.where((round(data["cc_count_may"]/(data["cc_count_may"]+data["dc_count_may"]),2)).isna(),0,(round(data["cc_count_may"]/(data["cc_count_may"]+data["dc_count_may"]),2)))
data["perc_cc_count_jun"] = np.where((round(data["cc_count_jun"]/(data["cc_count_jun"]+data["dc_count_jun"]),2)).isna(),0,(round(data["cc_count_jun"]/(data["cc_count_jun"]+data["dc_count_jun"]),2)))

data["perc_dc_count_apr"] = np.where((round(data["dc_count_apr"]/(data["cc_count_apr"]+data["dc_count_apr"]),2)).isna(),0,(round(data["dc_count_apr"]/(data["cc_count_apr"]+data["dc_count_apr"]),2)))
data["perc_dc_count_may"] = np.where((round(data["dc_count_may"]/(data["cc_count_may"]+data["dc_count_may"]),2)).isna(),0,(round(data["dc_count_may"]/(data["cc_count_may"]+data["dc_count_may"]),2)))
data["perc_dc_count_jun"] = np.where((round(data["dc_count_jun"]/(data["cc_count_jun"]+data["dc_count_jun"]),2)).isna(),0,(round(data["dc_count_jun"]/(data["cc_count_jun"]+data["dc_count_jun"]),2)))

data["perc_credit_amount_apr"] = np.where((round(data["credit_amount_apr"]/(data["credit_amount_apr"]+data["debit_amount_apr"]),2)).isna(),0,(round(data["credit_amount_apr"]/(data["credit_amount_apr"]+data["debit_amount_apr"]),2)))
data["perc_credit_amount_may"] = np.where((round(data["credit_amount_may"]/(data["credit_amount_may"]+data["debit_amount_may"]),2)).isna(),0,(round(data["credit_amount_may"]/(data["credit_amount_may"]+data["debit_amount_may"]),2)))
data["perc_credit_amount_jun"] = np.where((round(data["credit_amount_jun"]/(data["credit_amount_jun"]+data["debit_amount_jun"]),2)).isna(),0,(round(data["credit_amount_jun"]/(data["credit_amount_jun"]+data["debit_amount_jun"]),2)))

data["perc_debit_amount_apr"] = np.where((round(data["debit_amount_apr"]/(data["credit_amount_apr"]+data["debit_amount_apr"]),2)).isna(),0,(round(data["debit_amount_apr"]/(data["credit_amount_apr"]+data["debit_amount_apr"]),2)))
data["perc_debit_amount_may"] = np.where((round(data["debit_amount_may"]/(data["credit_amount_may"]+data["debit_amount_may"]),2)).isna(),0,(round(data["debit_amount_may"]/(data["credit_amount_may"]+data["debit_amount_may"]),2)))
data["perc_debit_amount_jun"] = np.where((round(data["debit_amount_jun"]/(data["credit_amount_jun"]+data["debit_amount_jun"]),2)).isna(),0,(round(data["debit_amount_jun"]/(data["credit_amount_jun"]+data["debit_amount_jun"]),2)))

data["perc_credit_count_apr"] = np.where((round(data["credit_count_apr"]/(data["credit_count_apr"]+data["debit_count_apr"]),2)).isna(),0,(round(data["credit_count_apr"]/(data["credit_count_apr"]+data["debit_count_apr"]),2)))
data["perc_credit_count_may"] = np.where((round(data["credit_count_may"]/(data["credit_count_may"]+data["debit_count_may"]),2)).isna(),0,(round(data["credit_count_may"]/(data["credit_count_may"]+data["debit_count_may"]),2)))
data["perc_credit_count_jun"] = np.where((round(data["credit_count_jun"]/(data["credit_count_jun"]+data["debit_count_jun"]),2)).isna(),0,(round(data["credit_count_jun"]/(data["credit_count_jun"]+data["debit_count_jun"]),2)))

data["perc_debit_count_apr"] = np.where((round(data["debit_count_apr"]/(data["credit_count_apr"]+data["debit_count_apr"]),2)).isna(),0,(round(data["debit_count_apr"]/(data["credit_count_apr"]+data["debit_count_apr"]),2)))
data["perc_debit_count_may"] = np.where((round(data["debit_count_may"]/(data["credit_count_may"]+data["debit_count_may"]),2)).isna(),0,(round(data["debit_count_may"]/(data["credit_count_may"]+data["debit_count_may"]),2)))
data["perc_debit_count_jun"] = np.where((round(data["debit_count_jun"]/(data["credit_count_jun"]+data["debit_count_jun"]),2)).isna(),0,(round(data["debit_count_jun"]/(data["credit_count_jun"]+data["debit_count_jun"]),2)))

data["cc_cons_exceed_apr"] = np.where((round((data["card_lim"]-data["cc_cons_apr"])/data["card_lim"],2)).isna(), 0 ,
                                      round((data["card_lim"]-data["cc_cons_apr"])/data["card_lim"],2))
data["cc_cons_exceed_may"] = np.where((round((data["card_lim"]-data["cc_cons_may"])/data["card_lim"],2)).isna(), 0 ,
                                      round((data["card_lim"]-data["cc_cons_may"])/data["card_lim"],2))
data["cc_cons_exceed_jun"] = np.where((round((data["card_lim"]-data["cc_cons_jun"])/data["card_lim"],2)).isna(), 0 ,
                                      round((data["card_lim"]-data["cc_cons_jun"])/data["card_lim"],2))
data["cc_cons_exceed_total"] = np.where((round(((data["card_lim"]*3)-data["total_cc_spend"])/(data["card_lim"]*3),2)).isna(), 0 ,
                                      round(((data["card_lim"]*3)-data["total_cc_spend"])/(data["card_lim"]*3),2))

data["credit_amount_exceed_apr"] = np.where((round((data["credit_amount_apr"]-data["cc_cons_apr"])/data["credit_amount_apr"],2)).isna(), 0 ,
                                      round((data["credit_amount_apr"]-data["cc_cons_apr"])/data["credit_amount_apr"],2))
data["credit_amount_exceed_may"] = np.where((round((data["credit_amount_may"]-data["cc_cons_may"])/data["credit_amount_may"],2)).isna(), 0 ,
                                      round((data["credit_amount_may"]-data["cc_cons_may"])/data["credit_amount_may"],2))
data["credit_amount_exceed_jun"] = np.where((round((data["credit_amount_jun"]-data["cc_cons_jun"])/data["credit_amount_jun"],2)).isna(), 0 ,
                                      round((data["credit_amount_jun"]-data["cc_cons_jun"])/data["credit_amount_jun"],2))
data["credit_amount_exceed_total"] = np.where((round(((data["credit_amount_jun"]+data["credit_amount_may"]+data["credit_amount_apr"])-data["total_cc_spend"])/(data["credit_amount_jun"]+data["credit_amount_may"]+data["credit_amount_apr"]),2)).isna(), 0 ,
                                      round(((data["credit_amount_jun"]+data["credit_amount_may"]+data["credit_amount_apr"])-data["total_cc_spend"])/(data["credit_amount_jun"]+data["credit_amount_may"]+data["credit_amount_apr"]),2))

data["avg_trend_cc_cons_exceed"] = (data["cc_cons_exceed_apr"]+data["cc_cons_exceed_may"]+data["cc_cons_exceed_jun"])/3
data["avg_trend_credit_amount_exceed"] = (data["credit_amount_exceed_apr"]+data["credit_amount_exceed_may"]+data["credit_amount_exceed_jun"])/3

data["avg_cc_spend_per_trans_apr"] = np.where((round(data["cc_cons_apr"]/data["cc_count_apr"],2)).isna(), 0, 
                                               round(data["cc_cons_apr"]/data["cc_count_apr"],2))
data["avg_cc_spend_per_trans_may"] = np.where((round(data["cc_cons_may"]/data["cc_count_may"],2)).isna(), 0, 
                                               round(data["cc_cons_may"]/data["cc_count_may"],2))
data["avg_cc_spend_per_trans_jun"] = np.where((round(data["cc_cons_jun"]/data["cc_count_jun"],2)).isna(), 0, 
                                               round(data["cc_cons_jun"]/data["cc_count_jun"],2))
data["avg_cc_spend_per_trans"] = np.where((round(data["total_cc_spend"]/data["total_cc_count"],2)).isna(), 0, 
                                               round(data["total_cc_spend"]/data["total_cc_count"],2))

data["est_cc_cons_3months"] = data["avg_cc_spend_per_trans"]*data["avg_credit_count_per_mnth"]*3
data["est_cc_cons_3months_exceed"] = data["est_cc_cons_3months"]*data["avg_trend_cc_cons_exceed"]

data["avg_dc_spend_per_trans_apr"] = np.where((round(data["dc_cons_apr"]/data["cc_count_apr"],2)).isna(), 0, 
                                               round(data["dc_cons_apr"]/data["cc_count_apr"],2))
data["avg_dc_spend_per_trans_may"] = np.where((round(data["dc_cons_may"]/data["cc_count_may"],2)).isna(), 0, 
                                               round(data["dc_cons_may"]/data["cc_count_may"],2))
data["avg_dc_spend_per_trans_jun"] = np.where((round(data["dc_cons_jun"]/data["cc_count_jun"],2)).isna(), 0, 
                                               round(data["dc_cons_jun"]/data["cc_count_jun"],2))
data["avg_dc_spend_per_trans"] = np.where((round(data["total_dc_spend"]/data["total_cc_count"],2)).isna(), 0, 
                                               round(data["total_dc_spend"]/data["total_cc_count"],2))

data["est_dc_cons_3months"] = data["avg_dc_spend_per_trans"]*data["avg_debit_count_per_mnth"]*3

data["est_tot_cons_3months"] = data["est_cc_cons_3months"]+data["est_dc_cons_3months"]
data["perc_est_cons_cc_dc"] = np.where((round(data["est_cc_cons_3months"]/data["est_dc_cons_3months"],2)).isna(),0,
                                      round(data["est_cc_cons_3months"]/data["est_dc_cons_3months"],2))
data["perc_est_cc_cons"] = np.where((round(data["est_cc_cons_3months"]/data["est_tot_cons_3months"],2)).isna(),0,
                                      round(data["est_cc_cons_3months"]/data["est_tot_cons_3months"],2))
data["perc_est_dc_cons"] = np.where((round(data["est_dc_cons_3months"]/data["est_tot_cons_3months"],2)).isna(),0,
                                      round(data["est_dc_cons_3months"]/data["est_tot_cons_3months"],2))

data["total_spend"] = data["total_cc_spend"] + data["total_dc_spend"]
data["total_count"] = data["total_cc_count"] + data["total_dc_count"]
data["total_spend_apr"] = data["cc_cons_apr"] + data["dc_cons_apr"]
data["total_count_apr"] = data["cc_count_apr"] + data["dc_count_apr"]
data["total_spend_may"] = data["cc_cons_may"] + data["dc_cons_may"]
data["total_count_may"] = data["cc_count_may"] + data["dc_count_may"]
data["total_spend_jun"] = data["cc_cons_jun"] + data["dc_cons_jun"]
data["total_count_jun"] = data["cc_count_jun"] + data["dc_count_jun"]

data["emi_total"] = data["emi_active"]*3
data["emi_perc_cc_spend_apr"] = np.where((round(data["emi_active"]/data["cc_cons_apr"],2)).isna(),0,round(data["emi_active"]/data["cc_cons_apr"],2))
data["emi_perc_cc_spend_may"] = np.where((round(data["emi_active"]/data["cc_cons_may"],2)).isna(),0,round(data["emi_active"]/data["cc_cons_may"],2))
data["emi_perc_cc_spend_jun"] = np.where((round(data["emi_active"]/data["cc_cons_jun"],2)).isna(),0,round(data["emi_active"]/data["cc_cons_jun"],2))
data["emi_perc_cc_spend"] = np.where((round(data["emi_total"]/data["total_cc_spend"],2)).isna(),0,round(data["emi_total"]/data["total_cc_spend"],2))

data["inv_perc_cc_spend"] = np.where((round(data["total_investment"]/data["cc_cons_jun"],2)).isna(),0,round(data["total_investment"]/data["cc_cons_jun"],2))
data["inv_perc_tot_spend"] = np.where((round(data["total_investment"]/data["total_spend_jun"],2)).isna(),0,round(data["total_investment"]/data["total_spend_jun"],2))


data["avg_credit_amount_apr"] = np.where((round(data["credit_amount_apr"]/data["credit_count_apr"],2)).isna(),0,
                                         round(data["credit_amount_apr"]/data["credit_count_apr"],2))
data["avg_credit_amount_may"] = np.where((round(data["credit_amount_may"]/data["credit_count_may"],2)).isna(),0,
                                         round(data["credit_amount_may"]/data["credit_count_may"],2))
data["avg_credit_amount_jun"] = np.where((round(data["credit_amount_jun"]/data["credit_count_jun"],2)).isna(),0,
                                         round(data["credit_amount_jun"]/data["credit_count_jun"],2))
data["avg_debit_amount_apr"] = np.where((round(data["debit_amount_apr"]/data["debit_count_apr"],2)).isna(),0,
                                         round(data["debit_amount_apr"]/data["debit_count_apr"],2))
data["avg_debit_amount_may"] = np.where((round(data["debit_amount_may"]/data["debit_count_may"],2)).isna(),0,
                                         round(data["debit_amount_may"]/data["debit_count_may"],2))
data["avg_debit_amount_jun"] = np.where((round(data["debit_amount_jun"]/data["debit_count_jun"],2)).isna(),0,
                                         round(data["debit_amount_jun"]/data["debit_count_jun"],2))
data["avg_debit_amount"] = np.where((round((data["debit_amount_apr"]+data["debit_amount_may"]+data["debit_amount_jun"])/(data["debit_count_apr"]+data["debit_count_may"]+data["debit_count_jun"]),2)).isna(),0,
                                         round((data["debit_amount_apr"]+data["debit_amount_may"]+data["debit_amount_jun"])/(data["debit_count_apr"]+data["debit_count_may"]+data["debit_count_jun"]),2))
data["avg_credit_amount"] = np.where((round((data["credit_amount_apr"]+data["credit_amount_may"]+data["credit_amount_jun"])/(data["credit_count_apr"]+data["credit_count_may"]+data["credit_count_jun"]),2)).isna(),0,
                                         round((data["credit_amount_apr"]+data["credit_amount_may"]+data["credit_amount_jun"])/(data["credit_count_apr"]+data["credit_count_may"]+data["credit_count_jun"]),2))
data["avg_exceed_cc_cardlim"] = np.where((round(data["avg_cc_spend_per_mnth"]/data["card_lim"],2)).isna(),0,round(data["avg_cc_spend_per_mnth"]/data["card_lim"],2))


data["cc_cardlim_apr"] = np.where((round(data["cc_cons_apr"]/data["card_lim"],2)).isna(),0,round(data["cc_cons_apr"]/data["card_lim"],2))
data["cc_cardlim_may"] = np.where((round(data["cc_cons_may"]/data["card_lim"],2)).isna(),0,round(data["cc_cons_may"]/data["card_lim"],2))
data["cc_cardlim_jun"] = np.where((round(data["cc_cons_jun"]/data["card_lim"],2)).isna(),0,round(data["cc_cons_jun"]/data["card_lim"],2))


data["cc_apr_may_spenddup"] = np.where((round((data["cc_cons_may"]-data["cc_cons_apr"])/data["cc_cons_apr"],2)).isna(), 0,
                                             round((data["cc_cons_may"]-data["cc_cons_apr"])/data["cc_cons_apr"],2))
data["cc_may_jun_spenddup"] = np.where((round((data["cc_cons_jun"]-data["cc_cons_may"])/data["cc_cons_may"],2)).isna(), 0,
                                    round((data["cc_cons_jun"]-data["cc_cons_may"])/data["cc_cons_may"],2))
data["dc_apr_may_spenddup"] = np.where((round((data["dc_cons_may"]-data["dc_cons_apr"])/data["dc_cons_apr"],2)).isna(), 0,
                                             round((data["dc_cons_may"]-data["dc_cons_apr"])/data["dc_cons_apr"],2))
data["dc_may_jun_spenddup"] = np.where((round((data["dc_cons_jun"]-data["dc_cons_may"])/data["dc_cons_may"],2)).isna(), 0,
                                             round((data["dc_cons_jun"]-data["dc_cons_may"])/data["dc_cons_may"],2))
data["cc_count_apr_maydup"] = np.where((round((data["cc_count_may"]-data["cc_count_apr"])/data["cc_count_apr"],2)).isna(), 0,
                                             round((data["cc_count_may"]-data["cc_count_apr"])/data["cc_count_apr"],2))
data["cc_count_may_jundup"] = np.where((round((data["cc_count_jun"]-data["cc_count_may"])/data["cc_count_may"],2)).isna(), 0,
                                             round((data["cc_count_jun"]-data["cc_count_may"])/data["cc_count_may"],2))
data["dc_count_apr_maydup"] = np.where((round((data["dc_count_may"]-data["dc_count_apr"])/data["dc_count_apr"],2)).isna(), 0,
                                             round((data["dc_count_may"]-data["dc_count_apr"])/data["dc_count_apr"],2))
data["dc_count_may_jundup"] = np.where((round((data["dc_count_jun"]-data["dc_count_may"])/data["dc_count_may"],2)).isna(), 0,
                                             round((data["dc_count_jun"]-data["dc_count_may"])/data["dc_count_may"],2))
data["debit_amount_apr_maydup"] = np.where((round((data["dc_count_jun"]-data["dc_count_may"])/data["dc_count_may"],2)).isna(), 0,
                                                 round((data["debit_amount_may"]-data["debit_amount_apr"])/data["debit_amount_apr"],2))
data["debit_amount_may_jundup"] = np.where((round((data["debit_amount_jun"]-data["debit_amount_may"])/data["debit_amount_may"],2)).isna(), 0,
                                                round((data["debit_amount_jun"]-data["debit_amount_may"])/data["debit_amount_may"],2))
data["credit_amount_apr_maydup"] = np.where((round((data["credit_amount_may"]-data["credit_amount_apr"])/data["credit_amount_apr"],2)), 0,
                                                  round((data["credit_amount_may"]-data["credit_amount_apr"])/data["credit_amount_apr"],2))
data["credit_amount_may_jundup"] = np.where((round((data["credit_amount_jun"]-data["credit_amount_may"])/data["credit_amount_may"],2)), 0,
                                                  round((data["credit_amount_jun"]-data["credit_amount_may"])/data["credit_amount_may"],2))
data["debit_count_apr_maydup"] = np.where((round((data["debit_count_may"]-data["debit_count_apr"])/data["debit_count_apr"],2)), 0,
                                                round((data["debit_count_may"]-data["debit_count_apr"])/data["debit_count_apr"],2))
data["debit_count_may_jundup"] = np.where((round((data["debit_count_jun"]-data["debit_count_may"])/data["debit_count_may"],2)), 0,
                                                round((data["debit_count_jun"]-data["debit_count_may"])/data["debit_count_may"],2))
data["credit_count_apr_maydup"] = np.where((round((data["credit_count_may"]-data["credit_count_apr"])/data["credit_count_apr"],2)), 0,
                                        round((data["credit_count_may"]-data["credit_count_apr"])/data["credit_count_apr"],2))
data["credit_count_may_jundup"] = np.where((round((data["credit_count_jun"]-data["credit_count_may"])/data["credit_count_may"],2)), 0,
                                                 round((data["credit_count_jun"]-data["credit_count_may"])/data["credit_count_may"],2))
data["max_credit_amount_apr_maydup"] = np.where((round((data["max_credit_amount_may"]-data["max_credit_amount_apr"])/data["max_credit_amount_apr"],2)), 0,
                                                      round((data["max_credit_amount_may"]-data["max_credit_amount_apr"])/data["max_credit_amount_apr"],2))
data["max_credit_amount_may_jundup"] = np.where((round((data["max_credit_amount_jun"]-data["max_credit_amount_may"])/data["max_credit_amount_may"],2)), 0,
                                                      round((data["max_credit_amount_jun"]-data["max_credit_amount_may"])/data["max_credit_amount_may"],2))


In [22]:
data.shape

(46887, 201)

In [23]:
## Replacing inf values with 0
data = data.replace([np.inf, -np.inf], np.nan)
for column in data.select_dtypes(include=np.number).columns:
    data[column] = data[column].fillna(0)

In [26]:
## Label encoding categorical variables
from sklearn.preprocessing import LabelEncoder

#Auto encodes any dataframe column of type category or object.
def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                df[feature] = le.fit_transform(df[feature])
            except:
                print('Error encoding '+feature)
        return df
    
data = dummyEncode(data)

In [41]:
## Splitting full data into train and test
train = data[data["Flag"]==1]
test = data[data["Flag"]==0]

train.drop('Flag', axis=1, inplace=True)
test.drop('Flag', axis=1, inplace=True)

test.drop('cc_cons', axis=1, inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [30]:
## Creating final datasets for training and testing
cols=["id","cc_cons"]
X = train.drop(cols,axis=1)
y = np.log(train["cc_cons"]+1)
X_test  = test.drop("id",axis=1)


Modeling Exercise starts here !!!

In [32]:
## Parameters for LightGBM
params = {'objective' : "regression", 
               'boost':"gbdt",
               'metric':"rmse",
               'boost_from_average':"false",
               'num_threads':8,
               'learning_rate' : 0.01,
               'num_leaves' : 13,
               'max_depth':-1,
               'tree_learner' : "serial",
               'feature_fraction' : 1,
               'bagging_freq' : 5,
               'bagging_fraction' : 0.5,
               #'min_data_in_leaf' : 80,
               #'min_sum_hessian_in_leaf' : 10.0,
               'verbosity' : 1}

In [33]:
## Running 10 folds cross validation 
from sklearn.model_selection import StratifiedKFold, KFold
def run_LGB(params, X, y, n_folds = 10, seed = 0):
    skf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
    #X, y = train[feature_names], train.amount_spent_per_room_night_scaled.values
    
    preds = np.zeros(test.shape[0])
    
    for i, (itr, icv) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[itr, :], X.iloc[icv, :]
        y_train, y_val = y[itr], y[icv]
        
        #X_train, y_train = balance_train_data3(X_train, y_train)
        #y_train = np.log(y_train+1)
        #y_val = np.log(y_val+1)
        dtrain = lgb.Dataset(X_train, y_train)
        dval = lgb.Dataset(X_val, y_val, reference=dtrain)

        bst = lgb.train(
                params                = params,
                train_set             = dtrain, 
                valid_sets            = [dtrain, dval],
                valid_names           = ['train', 'eval'],
                num_boost_round       = 10000,
                verbose_eval          = 500,
                early_stopping_rounds = 1000
            )

        score_, iter_ = bst.best_score['eval']['rmse'], bst.best_iteration
        test_preds = bst.predict(X_test, num_iteration = iter_)
        test_preds = np.exp(test_preds)-1
        #test_preds = np.square(np.exp(test_preds)-1)
        preds += test_preds
    return preds / n_folds

In [34]:
output = run_LGB(params, X,y)

Training until validation scores don't improve for 1000 rounds.
[500]	train's rmse: 1.07403	eval's rmse: 1.09601
[1000]	train's rmse: 1.04653	eval's rmse: 1.09342
[1500]	train's rmse: 1.02475	eval's rmse: 1.09357
Early stopping, best iteration is:
[970]	train's rmse: 1.04796	eval's rmse: 1.09308
Training until validation scores don't improve for 1000 rounds.
[500]	train's rmse: 1.07389	eval's rmse: 1.09264
[1000]	train's rmse: 1.04571	eval's rmse: 1.09132
[1500]	train's rmse: 1.02388	eval's rmse: 1.09253
Early stopping, best iteration is:
[715]	train's rmse: 1.0598	eval's rmse: 1.09069
Training until validation scores don't improve for 1000 rounds.
[500]	train's rmse: 1.07078	eval's rmse: 1.11716
[1000]	train's rmse: 1.04313	eval's rmse: 1.1147
[1500]	train's rmse: 1.02132	eval's rmse: 1.11507
Early stopping, best iteration is:
[754]	train's rmse: 1.05526	eval's rmse: 1.11401
Training until validation scores don't improve for 1000 rounds.
[500]	train's rmse: 1.07303	eval's rmse: 1.1052

In [38]:
## Writing the resutls to a dataframe
submission_1 = pd.DataFrame({
        "id": test_data["id"],
        "cc_cons": output
    })
submission_1.to_csv('submission_1.csv', index=False)

In [39]:
## Function to generate a download link to access the output file
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


# create a link to download the dataframe
create_download_link(submission_1)

In [ ]:
#from imblearn.over_sampling import RandomOverSampler
#from collections import Counter
#import random

#def balance_class_by_limited_over_sampling(X, y, max_sample_per_class=None, multiply_limit=7., random_state=42):
 #   """Balance class distribution basically by oversampling but limited duplication.
#
 #   # Arguments
  #      X: Data samples, only size of samples is used here.
   #     y: Class labels to be balanced.
    #    max_sample_per_class: Number of maximum samples per class, large class will be limitd to this number.
     #   multiply_limit: Small size class samples will be duplicated, but limited to multiple of this number.
    #"""
    #assert len(X) == len(y), f'Length of X({len(X)}) and y({len(y)}) are supposed to be the same.'
    #y_count = Counter(y)
    #max_sample_per_class = max_sample_per_class or np.max(list(y_count.values()))
    #resampled_idxes = []
    #random.seed(random_state)
    #for cur_y, count in y_count.items():
     #   this_samples = np.min([multiply_limit * count, max_sample_per_class]).astype(int)
     #   idxes = np.where(y == cur_y)[0]
     #   idxes = random.choices(idxes, k=this_samples)
     #   resampled_idxes += list(idxes)
    #return X[resampled_idxes], y[resampled_idxes]

#def balance_train_data3(X, y, random_state=42):
#    X_index = np.array(range(len(X)))
#    y_as_label = y.map(int)
#    X_resampled_index, _ = balance_class_by_limited_over_sampling(X_index, y_as_label, multiply_limit=2.)
#    X_resampled_index = sorted(X_resampled_index)
#    return X.iloc[X_resampled_index].reset_index(drop=True), y.iloc[X_resampled_index].reset_index(drop=True)

#X_train_scaled3, y_tr3 = balance_train_data3(X, y)
